# Clean Air Act Inspections Evaluations by Congressional District

This notebook examines ECHO data on air inspections, the means by which EPA and state environmental agencies check facilities for their compliance with the Clean Air Act. This notebook draws mainly on the ECHO downloadable file ICIS_FEC_EPA_INSPECTIONS. 
The fields in this table are:
<ol>
    <li>REGISTRY_ID, </li>
    <li>STATUTE_CODE, </li>
    <li>ACTUAL_BEGIN_DATE</li>

The REGISTRY_ID identifier is linked to the ECHO_EXPORTER table's field of the same name.

The STATUTE_CODE of interest for this notebook is 'CAA'.
    
**A state and congressional district must be chosen using the dropdown widgets that are provided.**

Note:  This notebook has widgets to select the state, congressional district and facility.  Those currently work only when you **run cells one at a time sequentially, not all at once.**

---
---

## How to Run
* If you click on a gray **code** cell, a little “play button” arrow appears on the left. If you click the play button, it will run the code in that cell (“**running** a cell”). The button will animate. When the animation stops, the cell has finished running.
![Where to click to run the cell](https://github.com/edgi-govdata-archiving/EEW-Image-Assets/blob/master/Jupyter%20instructions/pressplay.JPG?raw=true)
* You may get a warning that the notebook was not authored by Google. We know, we authored them! It’s okay. Click “Run Anyway” to continue. 
![Error Message](https://github.com/edgi-govdata-archiving/EEW-Image-Assets/blob/master/Jupyter%20instructions/warning-message.JPG?raw=true)
* **It is important to run cells in order because they depend on each other.**
* Some cells, like the one shown below, will create a dropdown menu after you run them. Be sure to make a selection (for example, click to change NY to LA) before running the next cell.
![Dropdown menu](https://github.com/edgi-govdata-archiving/EEW-Image-Assets/blob/master/Jupyter%20instructions/dropdown.JPG?raw=true)
* Other cells will simply print information when you run them, like this one:
![Simple cell](https://github.com/edgi-govdata-archiving/EEW-Image-Assets/blob/master/Jupyter%20instructions/cell-simple.JPG?raw=true)
* Run all of the cells in a Notebook to make a complete report. Please feel free to look at and **learn about each result as you create it**!

---
---

# Let's Begin!

### Hover over the "[ ]" on the top left corner of the cell below and you should see a "play" button appear. Click on it to run the cell then move to the next one.

In [1]:
# Import libraries
import urllib.parse
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import folium

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display


### Run this next cell to create the widget for selecting states. It will create a dropdown menu at the bottom. Choose your state from the menu then move on to the next cell.

In [2]:
states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]
dropdown_state=widgets.Dropdown(
    options=states,
    value='NY',
    description='State:',
    disabled=False,
)
output_state = widgets.Output()
my_state = ""

def dropdown_state_eventhandler( change ):
    output_state.clear_output()
    value = change.new
    with output_state:
        display( change.new )
            
dropdown_state.observe( dropdown_state_eventhandler, names='value')
display( dropdown_state )

Dropdown(description='State:', index=32, options=('AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', …

### Run this cell after choosing a state. It will pull the data for that state from ECHO

In [3]:
my_state = dropdown_state.value

sql = "select REGISTRY_ID, FAC_NAME, FAC_STATE, FAC_LAT, FAC_LONG, FAC_DERIVED_CD113 " + \
    " from ECHO_EXPORTER where AIR_FLAG = 'Y' and FAC_STATE = '" + my_state + "'"
url='http://apps.tlt.stonybrook.edu/echoepa/?query='
data_location=url+urllib.parse.quote(sql)
print(sql)
print(data_location)


select REGISTRY_ID, FAC_NAME, FAC_STATE, FAC_LAT, FAC_LONG, FAC_DERIVED_CD113  from ECHO_EXPORTER where AIR_FLAG = 'Y' and FAC_STATE = 'NY'
http://apps.tlt.stonybrook.edu/echoepa/?query=select%20REGISTRY_ID%2C%20FAC_NAME%2C%20FAC_STATE%2C%20FAC_LAT%2C%20FAC_LONG%2C%20FAC_DERIVED_CD113%20%20from%20ECHO_EXPORTER%20where%20AIR_FLAG%20%3D%20%27Y%27%20and%20FAC_STATE%20%3D%20%27NY%27


### Run this cell to load the CSV of that data. Then, we will be able to analyze it!

In [4]:
try:
    echo_data = pd.read_csv(data_location,encoding='iso-8859-1',header = 0)
    echo_data.set_index( "REGISTRY_ID", inplace=True)
except:
    print( "No data for " + my_state )

### How many facilities in the selected state are tracked under the Clean Air Act in the ICIS database?

In [5]:
num_facilities = echo_data.shape[0]
print("There are %s ICIS facilities in %s tracked in the ECHO database." %(num_facilities, my_state))

There are 12464 ICIS facilities in NY tracked in the ECHO database.


### Run this next cell to generate the Congressional District dropdown list for your state.  Then select a CD and run the following cell.
#### Here is a map of congressional districts: https://www.govtrack.us/congress/members/map

In [6]:
if (( my_state != 'none' ) & (my_state != 'all' )):
    cd_array = echo_data["FAC_DERIVED_CD113"].fillna(0).astype(int).unique()
    cd_array.sort()
    w2=widgets.Dropdown(
        options=cd_array,
        value=1,
        description='Congressional Districts:',
        disabled=False,
    )
    display(w2)

Dropdown(description='Congressional Districts:', index=1, options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1…

### Run this next cell after choosing a congressional district from the dropdown list.
#### You will then see the total number of ICIS (EPA's Integrated Compliance System) facilities that are being regulated in your district under the Clean Air Act.

In [7]:
my_cd = w2.value
my_cd_facs = echo_data[echo_data["FAC_DERIVED_CD113"].fillna(0).astype(int) == my_cd]
num_facilities = my_cd_facs.shape[0]    
print("There are %s ICIS facilities in %s district %s tracked in the ECHO database." %(num_facilities, my_state, my_cd))

There are 400 ICIS facilities in NY district 1 tracked in the ECHO database.


### Next run the next few cells to look up the compliance monitoring history for the facilities in the selected state and congressional district.  This step may take a while to run.

In [8]:
# sql = "select PGM_SYS_ID, STATE_EPA_FLAG, AGENCY_TYPE_DESC, COMP_MONITOR_DESC, ACTUAL_END_DATE from `ICIS-AIR_FCES_PCES` where STATE_CODE='" + my_state + "'"
url='http://apps.tlt.stonybrook.edu/echoepa/?query='

sql = "select REGISTRY_ID, STATUTE_CODE, ACTUAL_END_DATE from `ICIS_FEC_EPA_INSPECTIONS` " + \
    "where STATUTE_CODE = 'CAA'"
# sql = "show columns from `ICIS_FEC_EPA_INSPECTIONS`"
# sql = "show tables from slim_echoepa_a"
data_location=url+urllib.parse.quote(sql)
print(sql)
print(data_location)

select REGISTRY_ID, STATUTE_CODE, ACTUAL_END_DATE from `ICIS_FEC_EPA_INSPECTIONS` where STATUTE_CODE = 'CAA'
http://apps.tlt.stonybrook.edu/echoepa/?query=select%20REGISTRY_ID%2C%20STATUTE_CODE%2C%20ACTUAL_END_DATE%20from%20%60ICIS_FEC_EPA_INSPECTIONS%60%20where%20STATUTE_CODE%20%3D%20%27CAA%27


In [9]:
icis_data = pd.read_csv(data_location,encoding='iso-8859-1',header = 0)
icis_data.set_index( "REGISTRY_ID", inplace=True)

In [10]:
# Find the facility that matches the inspection, by REGISTRY_ID.

my_cd_icis = pd.DataFrame()
no_data_ids = []
# Look through all facilities in my CD.
for fac in my_cd_facs.itertuples():
    # breakpoint()
    fac_id = fac.Index
    try:
        air_inspections = icis_data.loc[ fac_id ].copy().to_frame()
        # breakpoint()
        n = air_inspections.shape[0]
        fac_list = [fac.Index] * n
        air_inspections['facility'] = fac_list
        frames = [my_cd_icis, air_inspections]
        my_cd_icis = pd.concat( frames, ignore_index=False )
    except KeyError:
        # The facility wasn't found in the ICIS_FEC_EPA_INSPECTIONS data.
        no_data_ids.append( fac.Index )
    
# my_cd_icis.to_csv( r'my_cd_icis.csv', index=True, header=True )
# with open( "no_data_ids.txt", "w" ) as output:
#     output.write( str( no_data_ids ))

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  app.launch_new_instance()


### How many facilities have had compliance evaluations, over all years contained in the ECHO data?

In [11]:

print( "Facilities without inspections: " + str( len( no_data_ids )))
print( "Facilities with inspections: " + str( len( my_cd_icis )))

Facilities without inspections: 397
Facilities with inspections: 6


### Make a table that lists the names, locations, and IDs of the facilities that have received inspections

In [12]:
fac_idx_list = my_cd_icis['facility'].dropna()
fac_cd_icis = my_cd_facs.loc[fac_idx_list]
my_cd_facs.loc[fac_idx_list]

,FAC_NAME,FAC_STATE,FAC_LAT,FAC_LONG,FAC_DERIVED_CD113
REGISTRY_ID,,,,,
1.100008e+11,NORTHVILLE HOLTSVILLE TERMINAL,NY,40.812503,-73.065525,1.0
1.100008e+11,NORTHVILLE HOLTSVILLE TERMINAL,NY,40.812503,-73.065525,1.0
1.100195e+11,BROOKHAVEN LANDFILL & RECYCLING AREA,NY,40.797178,-72.930442,1.0
1.100195e+11,BROOKHAVEN LANDFILL & RECYCLING AREA,NY,40.797178,-72.930442,1.0
1.100016e+11,IRS BROOKHAVEN SERVICE CENTER,NY,40.816680,-73.045030,1.0
1.100016e+11,IRS BROOKHAVEN SERVICE CENTER,NY,40.816680,-73.045030,1.0


## Use that table to make an interactive map of facilities that have received inspections
### Once you run this cell, a map should appear. You can zoom in and out, or click on facilities to get their names.

In [13]:
# Let's show a quick map of your area and the facilities in it

def mapper(df):
    # Initialize the map
    m = folium.Map(
        location = [df.mean()["FAC_LAT"], df.mean()["FAC_LONG"]],
        zoom_start = 9
    )

    # Add a clickable marker for each facility
    for index, row in df.iterrows():
        folium.Marker(
            location = [row["FAC_LAT"], row["FAC_LONG"]],
            popup = row["FAC_NAME"] ).add_to(m)

    # Show the map
    return m

map_of_facilities_in_my_area = mapper(fac_cd_icis)
map_of_facilities_in_my_area

## Inspections by facility
### Run the below cell, then choose a facility from the dropdown that appears to delve deeper into inspections at that facility.

In [ ]:
cd_array = fac_cd_icis["FAC_NAME"].unique()
cd_array.sort()

w3=widgets.Dropdown(
    options=cd_array,
    description='Facility Name:',
    disabled=False,
)
display(w3)

### Run this next cell after choosing a facility to print out some details about it

In [ ]:
# This cell creates a function that will be used by both the CD and the state
# to plot the number of cases by year.
import datetime

def show_plot( df, date_field, year_field, place, date_format, chart_title ):
    format_str = date_format # The format
    nan_count = 0
    year_col = []
    for day in df[date_field]:
        try:
            # breakpoint()
            viol_year = datetime.datetime.strptime(day, format_str).year
            year_col.append( viol_year )
        except:
            nan_count += 1
            year_col.append(  np.NaN )
    df[year_field] = year_col
    
    year_groups = df.groupby( year_field )[[ year_field ]]
    counted_years = year_groups.count()

    # Print how many values are present 
    print(counted_years)
    chart_title +=  " in " + place + " by year"

    ax = counted_years[[year_field]].plot(kind='bar', title = chart_title, figsize=(15, 10), legend=False, fontsize=12)
    ax.set_xlabel("Year", fontsize=12)
    ax.set_ylabel("Count", fontsize=12)
    
my_fac = fac_cd_icis[fac_cd_icis["FAC_NAME"] == w3.value]
my_inspections = my_cd_icis[my_cd_icis['facility'] == my_fac.iloc[[0]].index[0]]
my_inspections

## This section saves some of this data to a CSV file in your Google Drive.
The first of the next three cells will open our Google Drive to write into.
The second cell writes the congressional district file.
The third cell writes the file for state data.
**Running these cells is optional.**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Write the congressional district data to CSV file.

In [ ]:
filename = '/content/drive/My Drive/cd-air-inspections-' + my_state + '-' + str( my_cd ) + '.csv'
my_cd_icis.to_csv( filename ) 
print( "Writing this data to %s" %(filename))

Write the state data to CSV file.

In [ ]:
filename = '/content/drive/My Drive/state-air-inspections-' + my_state + '.csv'
icis_data.to_csv( filename ) 
print( "Writing this data to %s" %(filename))

## Plot the number of inspections by year for the congressional district.

In [ ]:
chart_title = "Total CAA inspections"

show_plot( my_cd_icis, 'ACTUAL_END_DATE', 'ACTUAL_END_YEAR', \
          my_state + ' - #' + str( my_cd ), '%m/%d/%Y', chart_title )

## Plot the number of inspections by year, using the entire state.
Since the number of inspections in a single CD may be small, it can be more
interesting to look at the entire state.

In [ ]:
chart_title = "Total CAA inspections"

show_plot( icis_data, 'ACTUAL_END_DATE', 'ACTUAL_END_YEAR', my_state, \
          '%m/%d/%Y', chart_title )